In [16]:
import os
import pandas as pd
from binance.client import Client
from binance import ThreadedWebsocketManager
from datetime import datetime, timedelta


In [4]:
API_KEY = os.getenv('TEST_KEY')
API_SECRET = os.getenv('TEST_SECRET')
API_KEY

'3aH1Gh3xf1fsOxDEPpDERmBZ1NXSryFeWx3obCBDD3JveNIvoKHWKRvW3AktNnS2'

In [5]:
# Ensure that testnet is set to true !
client = Client(api_key = API_KEY, api_secret = API_SECRET, tld = "us", testnet= True)

In [6]:
client.get_account() #Test account details

{'makerCommission': 0,
 'takerCommission': 0,
 'buyerCommission': 0,
 'sellerCommission': 0,
 'commissionRates': {'maker': '0.00000000',
  'taker': '0.00000000',
  'buyer': '0.00000000',
  'seller': '0.00000000'},
 'canTrade': True,
 'canWithdraw': False,
 'canDeposit': False,
 'brokered': False,
 'requireSelfTradePrevention': False,
 'updateTime': 1672685531162,
 'accountType': 'SPOT',
 'balances': [{'asset': 'BNB',
   'free': '1000.00000000',
   'locked': '0.00000000'},
  {'asset': 'BTC', 'free': '1.00000000', 'locked': '0.00000000'},
  {'asset': 'BUSD', 'free': '10000.00000000', 'locked': '0.00000000'},
  {'asset': 'ETH', 'free': '100.00000000', 'locked': '0.00000000'},
  {'asset': 'LTC', 'free': '500.00000000', 'locked': '0.00000000'},
  {'asset': 'TRX', 'free': '500000.00000000', 'locked': '0.00000000'},
  {'asset': 'USDT', 'free': '10000.00000000', 'locked': '0.00000000'},
  {'asset': 'XRP', 'free': '50000.00000000', 'locked': '0.00000000'}],
 'permissions': ['SPOT']}

In [7]:
account = client.get_account()
account["accountType"]  #Account Type

'SPOT'

In [17]:
def get_history(symbol, interval, start, end = None):
        bars = client.get_historical_klines(symbol = symbol, interval = interval,
                                            start_str = start, end_str = end, limit = 1000)  # get the historical Klines with defined intervals 
        df = pd.DataFrame(bars)
        df["Date"] = pd.to_datetime(df.iloc[:,0], unit = "ms") #make the first column of unix time into meaningful timestamps
        df.columns = ["Open Time", "Open", "High", "Low", "Close", "Volume", "Close Time", "Quote Asset Volume", 
                      "Number of Trades", "Taker Buy Base Asset Volume", "Taker Buy Quote Asset Volume", "Ignore", "Date"]  # rename columns for what we are going to use them
        df = df[["Date", "Open", "High", "Low", "Close", "Volume"]].copy() #take only the columns we care about
        df.set_index("Date", inplace = True) # make the date column the index column
        for column in df.columns:
            df[column] = pd.to_numeric(df[column], errors = "coerce")
            
        return df

In [18]:
df = pd.DataFrame(account["balances"]) # extract balances from account
df

,asset,free,locked
0,BNB,1000.00000000,0.00000000
1,BTC,1.00000000,0.00000000
2,BUSD,10000.00000000,0.00000000
3,ETH,100.00000000,0.00000000
4,LTC,500.00000000,0.00000000
5,TRX,500000.00000000,0.00000000
6,USDT,10000.00000000,0.00000000
7,XRP,50000.00000000,0.00000000


In [19]:
# Convert numbers to floats
df["free"] = pd.to_numeric(df["free"], errors = "coerce")
df["locked"] = pd.to_numeric(df["locked"], errors = "coerce")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   asset   8 non-null      object 
 1   free    8 non-null      float64
 2   locked  8 non-null      float64
dtypes: float64(2), object(1)
memory usage: 320.0+ bytes


In [20]:
#Get prices for all testnet pairs
client.get_all_tickers()

[{'symbol': 'BNBBUSD', 'price': '245.90000000'},
 {'symbol': 'BTCBUSD', 'price': '16693.91000000'},
 {'symbol': 'ETHBUSD', 'price': '1215.77000000'},
 {'symbol': 'LTCBUSD', 'price': '75.10000000'},
 {'symbol': 'TRXBUSD', 'price': '0.05514000'},
 {'symbol': 'XRPBUSD', 'price': '0.34570000'},
 {'symbol': 'BNBUSDT', 'price': '246.09000000'},
 {'symbol': 'BTCUSDT', 'price': '16699.52000000'},
 {'symbol': 'ETHUSDT', 'price': '1216.28000000'},
 {'symbol': 'LTCUSDT', 'price': '75.12000000'},
 {'symbol': 'TRXUSDT', 'price': '0.05515000'},
 {'symbol': 'XRPUSDT', 'price': '0.34590000'},
 {'symbol': 'BNBBTC', 'price': '0.01473100'},
 {'symbol': 'ETHBTC', 'price': '0.07281900'},
 {'symbol': 'LTCBTC', 'price': '0.00449900'},
 {'symbol': 'TRXBTC', 'price': '0.00000331'},
 {'symbol': 'XRPBTC', 'price': '0.00002072'},
 {'symbol': 'LTCBNB', 'price': '0.30570000'},
 {'symbol': 'TRXBNB', 'price': '0.00022420'},
 {'symbol': 'XRPBNB', 'price': '0.00140700'}]

In [21]:
last24 = client.get_ticker(symbol = "BTCUSDT") #24 Price change statistic
last24

{'symbol': 'BTCUSDT',
 'priceChange': '111.47000000',
 'priceChangePercent': '0.672',
 'weightedAvgPrice': '16691.76655215',
 'prevClosePrice': '16587.83000000',
 'lastPrice': '16699.06000000',
 'lastQty': '0.00100000',
 'bidPrice': '16699.06000000',
 'bidQty': '0.01851000',
 'askPrice': '16699.30000000',
 'askQty': '0.17006800',
 'openPrice': '16587.59000000',
 'highPrice': '17898.53000000',
 'lowPrice': '8290.64000000',
 'volume': '7304.21126000',
 'quoteVolume': '121920189.19948064',
 'openTime': 1672621154597,
 'closeTime': 1672707554597,
 'firstId': 2149033,
 'lastId': 2256063,
 'count': 107031}

In [22]:
pd.to_datetime(last24["openTime"], unit = "ms") # Show datetime in a readable format

Timestamp('2023-01-02 00:59:14.597000')

In [23]:
timestamp = client._get_earliest_valid_timestamp(symbol = "BTCUSDT", interval = "1d") #earliest available data will reset every month
pd.to_datetime(timestamp, unit = "ms") # Show datetime in a readable format

Timestamp('2022-12-14 00:00:00')

In [27]:
df = get_history(symbol = "BTCUSDT", interval = "1d", start = timestamp)
df

,Open,High,Low,Close,Volume
Date,,,,,
2022-12-14,17837.75,50000.00,11960.00,17795.18,4428.098892
2022-12-15,17795.18,18952.74,16831.78,17357.11,7740.685182
2022-12-16,17357.49,17525.00,16342.69,16630.42,8635.080110
2022-12-17,16630.72,17376.90,5312.94,16776.19,7435.010972
2022-12-18,16776.61,17134.31,5000.00,16738.62,6363.810820
2022-12-19,16738.61,16907.00,10000.00,16439.51,7505.842789
2022-12-20,16439.51,17055.56,6000.00,16896.24,8441.694337
2022-12-21,16896.19,17095.64,5000.00,16826.39,6965.755353
2022-12-22,16825.18,16867.03,4001.00,16821.02,7266.582324


In [33]:
def stream_data(msg):
    '''define how to process incoming websocket messages'''
    time = pd.to_datetime(msg["E"], unit="ms")
    price = msg["c"]
    
    print("Time: {} | Price: {}".format(time, price))

In [29]:
#initialize and start the websocket
twm = ThreadedWebsocketManager(tld = "us")
twm.start()

In [34]:
twm.start_symbol_miniticker_socket(callback = stream_data, symbol = "BTCUSD")

'btcusd@miniTicker'

In [32]:
twm.stop() #stop the websocket/stream